In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('EmailSpan').getOrCreate()

In [17]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer,IDF,CountVectorizer,StringIndexer,StopWordsRemover
from pyspark.sql.functions import col,udf,length
from pyspark.sql.types import IntegerType


In [29]:
data=spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [30]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [31]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [32]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [33]:
data = data.withColumn('lenght',length(data['text']))

In [34]:
data.show()

+-----+--------------------+------+
|class|                text|lenght|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [35]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(lenght)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [36]:
indexer = StringIndexer(inputCol='class',outputCol='label')

In [37]:
#index_model = indexer.fit(data)

In [38]:
#index_data = index_model.transform(data)

In [39]:
tokenizer = Tokenizer(inputCol='text',outputCol='tok_word')

In [40]:
#tokenized_data = tokenizer.transform(index_data)

In [41]:
stop_word = StopWordsRemover(inputCol='tok_word',outputCol='stop_token')

In [42]:
#stop_data = stop_word.transform(tokenized_data)

In [43]:
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')

In [44]:
idf = IDF(inputCol='c_vec',outputCol='tf_idf')

In [45]:
from pyspark.ml.feature import VectorAssembler

In [46]:
clean_up = VectorAssembler(inputCols=['tf_idf','lenght'],outputCol='features')

In [47]:
from pyspark.ml.classification import NaiveBayes

In [48]:
nb = NaiveBayes()

In [49]:
from pyspark.ml import Pipeline

In [50]:
data_prep_pipe = Pipeline(stages=[indexer,tokenizer,stop_word,count_vec,idf,clean_up])

In [51]:
cleaner = data_prep_pipe.fit(data)

In [52]:
clean_data = cleaner.transform(data)

In [54]:
clean_data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lenght: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- tok_word: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)



In [55]:
final_data = clean_data.select('label','features')

In [56]:
training_data,test_data = final_data.randomSplit([0.7,0.3])

In [57]:
spam_model = nb.fit(training_data)

In [58]:
test_result= spam_model.transform(test_data)

In [59]:
test_result.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [61]:
m_eval = MulticlassClassificationEvaluator()

In [62]:
m_eval.evaluate(test_result)

0.9392990209262231